In [4]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")
chat = ChatOpenAI(model_name="gpt-3.5-turbo")

a = llm.predict("How many planets are there in the solar system?")
b = chat.predict("How many planets are there in the solar system?")

a, b

('\n\nThere are 8 planets in the solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'There are 8 planets in the solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

In [5]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
	SystemMessage(content="You are a professional italian chef."),
	AIMessage(content="Welcome to the italian restaurant. What would you like to order?"),
	HumanMessage(content="I'd like to order a pizza. What are the toppings?"),
]

chat.predict_messages(messages)

AIMessage(content='We have a variety of toppings available for our pizzas. Some popular options include classic Margherita with tomato sauce, fresh mozzarella, and basil, or a Quattro Stagioni with artichokes, mushrooms, ham, and olives. We also have options like Prosciutto e Rucola with prosciutto, arugula, and shaved Parmesan, or a Diavola with spicy salami and chili flakes. Feel free to customize your pizza with any toppings you prefer!')

In [6]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template("What is the distance between the {planet_a} and the {planet_b}?",)

prompt = template.format(planet_a="Earth", planet_b="Mars")

chat.predict(prompt)

'The distance between Earth and Mars varies depending on their positions in their respective orbits. On average, the distance between Earth and Mars is about 225 million kilometers (140 million miles). However, this distance can range from about 54.6 million kilometers (33.9 million miles) when the two planets are at their closest approach (opposition) to about 401 million kilometers (249 million miles) when they are at their farthest apart (conjunction).'

In [7]:
template = ChatPromptTemplate.from_messages([
	("system", "You are a professional {nationality} chef."),
	("ai", "Welcome to the {nationality} restaurant. What would you like to order?"),
	("human", "I'd like to order a {food}. What are the ingredients?"),
])

prompt = template.format_messages(nationality="italian", food="pizza")

chat.predict_messages(prompt)

AIMessage(content='For a classic Italian pizza, we typically use a thin crust topped with tomato sauce, mozzarella cheese, and various toppings such as pepperoni, mushrooms, bell peppers, onions, olives, and basil. Is there a specific type of pizza you have in mind or any dietary restrictions I should be aware of?')

In [8]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
	def parse(self, text: str) -> str:
		items = text.strip().split(",")
		return list(map(str.strip, items))

p = CommaOutputParser()

p.parse("Hello,how,are,you?")

['Hello', 'how', 'are', 'you?']

In [12]:
template = ChatPromptTemplate.from_messages([
	("system", """You are a list generating machine. 
	Everything you are asked will be answered with a comma separated list of max {max_items}, in lowercase. Do not reply with anything else.
	"""),
	("human", "{question}")
])

prompt = template.format_messages(
	max_items=10,
	question="What are the colors?"
)

output = chat.predict_messages(prompt)

p.parse(output.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'black',
 'white',
 'pink',
 'brown']

In [14]:
chain = template | chat | CommaOutputParser()

chain.invoke({
	"max_items": 10,
	"question": "What are the colors?"
})


['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [19]:
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
	model_name="gpt-3.5-turbo",
	temperature=0.1, 
	streaming=True, 
	callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
	("system", "You are a professional international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
	("human", "I want to cook a {cuisine} dish. What is the best recipe for beginner?"),
])

chef_chain = chef_prompt | chat

In [20]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
	("system", """
		You are a vegetarian chef specialized on making traditional recipes vegetarian. 
		You find alternative ingredients and explain their preparation. 
		You don't radically modify the recipe. 
		If there is no alternative for a food just say you don't know how to replace it
	"""),
	("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe" : chef_chain} | veg_chain

final_chain.invoke({
	"cuisine": "korean",
})


One of the best Korean dishes for beginners to try is Bibimbap. It's a delicious and colorful mixed rice dish that is easy to make and can be customized with your favorite ingredients. Here's a simple Bibimbap recipe for you to try:

Ingredients:
- 1 cup cooked white rice
- 1 carrot, julienned
- 1 zucchini, julienned
- 1 cup spinach
- 1 cup bean sprouts
- 2 eggs
- 2 cloves garlic, minced
- 2 tbsp soy sauce
- 1 tbsp sesame oil
- 1 tbsp vegetable oil
- Salt and pepper to taste
- Gochujang (Korean chili paste), to taste (optional)
- Sesame seeds, for garnish

Instructions:
1. Cook the rice according to package instructions and set aside.
2. In a small bowl, mix together the soy sauce, sesame oil, and minced garlic to make the sauce.
3. Heat vegetable oil in a pan over medium heat. Add the carrots and zucchini, and stir-fry for 2-3 minutes until slightly softened. Season with salt and pepper. Remove from the pan and set aside.
4. In the same pan, add the spinach and bean sprouts. Stir-fry 

AIMessageChunk(content="For a vegetarian version of Bibimbap, you can make a few simple ingredient swaps:\n\n1. **Eggs**: You can replace the eggs with a vegan alternative such as scrambled tofu or a vegan egg substitute. To make scrambled tofu, crumble firm tofu in a pan and season it with a bit of turmeric for color and salt and pepper for flavor.\n\n2. **Soy Sauce**: Make sure to use a vegetarian or vegan soy sauce to keep the dish vegetarian-friendly.\n\n3. **Gochujang (Korean chili paste)**: Most Gochujang brands are vegetarian, but it's always good to double-check the ingredients. If you can't find a vegetarian version, you can substitute it with a mixture of sriracha and miso paste for a similar flavor profile.\n\n4. **Sesame Oil**: Sesame oil is a key flavor in Bibimbap, but if you don't have it or prefer not to use it, you can substitute it with a neutral oil like grapeseed or avocado oil. You can also add a few drops of toasted sesame oil for a similar nutty flavor.\n\n5. **V

In [24]:
# Few-shot learning is a technique that allows you to train a model on a small amount of data.
from langchain.prompts.few_shot import FewShotPromptTemplate

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]		

example_prompt = """
  Human: {question}
	AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_prompt)

prompt = FewShotPromptTemplate(
	example_prompt=example_prompt, 
	examples=examples,
	suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({
    "country": "Finland"
})

AI: 
        Here is what I know:
        Capital: Helsinki
        Language: Finnish, Swedish
        Food: Rye bread, fish, berries
        Currency: Euro

AIMessageChunk(content='AI: \n        Here is what I know:\n        Capital: Helsinki\n        Language: Finnish, Swedish\n        Food: Rye bread, fish, berries\n        Currency: Euro')

In [27]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

examples = [
    {
        "country": "France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]		

example_prompt = ChatPromptTemplate.from_messages([
	("human", "What do you know about {country}?"),
	("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
	example_prompt=example_prompt,
	examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
	("system", "You are a geography expert."),
	example_prompt, # Put the few-shot prompt here
	("human", "What do you know about {country}?"),
])

chain = final_prompt | chat

chain.invoke({"country": "Turkey"})



        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebabs and Baklava
        Currency: Turkish Lira
        

AIMessageChunk(content='\n        I know this:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebabs and Baklava\n        Currency: Turkish Lira\n        ')